In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

class KEPCODataAnalyzer:
    def __init__(self):
        self.customer_data = None
        self.lp_data = None
        
    def load_customer_data(self, file_path):
        """고객 기본정보 로딩 및 기본 분석"""
        print("=== 고객 기본정보 로딩 ===")
        
        # Excel 파일 읽기 (실제 환경에서는 이 부분 사용)
        # self.customer_data = pd.read_excel(file_path)
        
        # 샘플 데이터 생성 (테스트용)
        self.customer_data = self._create_sample_customer_data()
        
        print(f"총 고객 수: {len(self.customer_data):,}명")
        print(f"컬럼: {list(self.customer_data.columns)}")
        print("\n기본 정보:")
        print(self.customer_data.head())
        
        return self._analyze_customer_distribution()
    
    def _create_sample_customer_data(self):
        """테스트용 샘플 고객 데이터 생성"""
        data = []
        contract_types = ['222 일반용(갑)‖고압A', '226 일반용(을) 고압A', '322 산업용(갑)‖고압A', '726 산업용(을) 고압A']
        usage_types = ['02 상업용', '09 광공업용']
        power_ranges = ['100~199', '200~299', '400~499', '500~599', '700~799', '800~899']
        industries = ['병원', '교회', '상가', 'CNG충전', '금속가공', '점포', '온천탕', '제조업', '마트']
        
        for i in range(1, 201):  # 200명
            data.append({
                '순번': i,
                '고객번호': f'A{1000+i}',
                '계약전력': np.random.choice(power_ranges),
                '계약종별': np.random.choice(contract_types),
                '사용용도': np.random.choice(usage_types),
                '주생산품': np.random.choice(industries),
                '산업분류': f'{np.random.randint(100,999)}기타업종'
            })
        
        return pd.DataFrame(data)
    
    def _analyze_customer_distribution(self):
        """고객 분포 분석"""
        print("\n=== 고객 분포 분석 ===")
        
        # 계약종별 분포
        contract_dist = self.customer_data['계약종별'].value_counts()
        print("\n📊 계약종별 분포:")
        for contract, count in contract_dist.items():
            print(f"  {contract}: {count}명 ({count/len(self.customer_data)*100:.1f}%)")
        
        # 사용용도별 분포
        usage_dist = self.customer_data['사용용도'].value_counts()
        print("\n🏭 사용용도별 분포:")
        for usage, count in usage_dist.items():
            print(f"  {usage}: {count}명 ({count/len(self.customer_data)*100:.1f}%)")
        
        # 계약전력 분포
        power_dist = self.customer_data['계약전력'].value_counts().sort_index()
        print("\n⚡ 계약전력 분포:")
        for power, count in power_dist.items():
            print(f"  {power}kW: {count}명 ({count/len(self.customer_data)*100:.1f}%)")
        
        return {
            'contract_distribution': contract_dist,
            'usage_distribution': usage_dist,
            'power_distribution': power_dist
        }
    
    def load_lp_data(self, file_paths):
        """LP 데이터 로딩 및 결합"""
        print("\n=== LP 데이터 로딩 ===")
        
        lp_dataframes = []
        
        for i, file_path in enumerate(file_paths, 1):
            print(f"파일 {i} 로딩: {file_path}")
            
            # CSV 파일 읽기 (실제 환경에서는 이 부분 사용)
            # df = pd.read_csv(file_path, encoding='utf-8')
            
            # 샘플 데이터 생성 (테스트용)
            df = self._create_sample_lp_data(i)
            
            print(f"  레코드 수: {len(df):,}")
            print(f"  고객 수: {df['대체고객번호'].nunique()}")
            print(f"  기간: {df['LP수신일자'].min()} ~ {df['LP수신일자'].max()}")
            
            lp_dataframes.append(df)
        
        # 데이터 결합
        self.lp_data = pd.concat(lp_dataframes, ignore_index=True)
        print(f"\n✅ 전체 LP 데이터 결합 완료:")
        print(f"  총 레코드: {len(self.lp_data):,}")
        print(f"  총 고객: {self.lp_data['대체고객번호'].nunique()}")
        
        return self._analyze_lp_quality()
    
    def _create_sample_lp_data(self, file_num):
        """테스트용 샘플 LP 데이터 생성"""
        data = []
        customers = [f'A{1001+i}' for i in range(10)]  # A1001~A1010
        
        # 파일별로 다른 기간 설정
        if file_num == 1:
            start_date = datetime(2024, 3, 1)
            days = 15
        else:
            start_date = datetime(2024, 3, 16) 
            days = 16
        
        for customer in customers:
            for day in range(days):
                current_date = start_date + timedelta(days=day)
                for hour in range(24):
                    for minute in [0, 15, 30, 45]:
                        timestamp = current_date.replace(hour=hour, minute=minute)
                        
                        # 시간대별 패턴을 반영한 가상 데이터
                        base_power = 80 + 30 * np.sin(2 * np.pi * hour / 24) + np.random.normal(0, 10)
                        base_power = max(0, base_power)
                        
                        data.append({
                            '대체고객번호': customer,
                            'LP수신일자': timestamp.strftime('%Y-%m-%d-%H:%M'),
                            '순방향유효전력': round(base_power + np.random.normal(0, 5), 1),
                            '지상무효': round(abs(np.random.normal(10, 5)), 1),
                            '진상무효': round(abs(np.random.normal(5, 3)), 1),
                            '피상전력': round(base_power + np.random.normal(0, 3), 1)
                        })
        
        return pd.DataFrame(data)
    
    def _analyze_lp_quality(self):
        """LP 데이터 품질 분석"""
        print("\n=== LP 데이터 품질 분석 ===")
        
        # 기본 통계
        print("📈 기본 통계:")
        numeric_cols = ['순방향유효전력', '지상무효', '진상무효', '피상전력']
        print(self.lp_data[numeric_cols].describe())
        
        # 시간 간격 체크
        print("\n⏰ 시간 간격 체크:")
        self.lp_data['LP수신일자_dt'] = pd.to_datetime(self.lp_data['LP수신일자'], format='%Y-%m-%d-%H:%M')
        
        for customer in self.lp_data['대체고객번호'].unique()[:3]:  # 처음 3명만 체크
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer].sort_values('LP수신일자_dt')
            time_diffs = customer_data['LP수신일자_dt'].diff().dt.total_seconds() / 60  # 분 단위
            
            print(f"  {customer}: 평균 간격 {time_diffs.mean():.1f}분, 표준편차 {time_diffs.std():.1f}분")
            
            # 15분이 아닌 간격 찾기
            non_15min = time_diffs[(time_diffs != 15.0) & (~time_diffs.isna())]
            if len(non_15min) > 0:
                print(f"    ⚠️ 비정상 간격: {len(non_15min)}건")
        
        # 결측치 및 이상치 체크
        print("\n🔍 데이터 품질 체크:")
        for col in numeric_cols:
            null_count = self.lp_data[col].isnull().sum()
            zero_count = (self.lp_data[col] == 0).sum()
            negative_count = (self.lp_data[col] < 0).sum()
            
            print(f"  {col}:")
            print(f"    결측치: {null_count:,}건 ({null_count/len(self.lp_data)*100:.2f}%)")
            print(f"    0값: {zero_count:,}건 ({zero_count/len(self.lp_data)*100:.2f}%)")
            print(f"    음수: {negative_count:,}건 ({negative_count/len(self.lp_data)*100:.2f}%)")
        
        # 고객별 데이터 완정성
        print("\n👥 고객별 데이터 완정성:")
        customer_counts = self.lp_data['대체고객번호'].value_counts()
        expected_records = 24 * 4 * 31  # 15분 간격 × 1개월
        
        print(f"  예상 레코드 수: {expected_records:,}개/고객")
        print(f"  실제 레코드 수: {customer_counts.min():,}~{customer_counts.max():,}개/고객")
        
        incomplete_customers = customer_counts[customer_counts < expected_records * 0.95]  # 95% 미만
        if len(incomplete_customers) > 0:
            print(f"  ⚠️ 불완전한 고객: {len(incomplete_customers)}명")
        else:
            print("  ✅ 모든 고객 데이터 완정성 양호")
        
        return {
            'total_records': len(self.lp_data),
            'customers': self.lp_data['대체고객번호'].nunique(),
            'date_range': (self.lp_data['LP수신일자_dt'].min(), self.lp_data['LP수신일자_dt'].max()),
            'data_quality': {col: {'null': self.lp_data[col].isnull().sum(), 
                                  'zero': (self.lp_data[col] == 0).sum(),
                                  'negative': (self.lp_data[col] < 0).sum()} for col in numeric_cols}
        }
    
    def detect_outliers(self, method='iqr'):
        """이상치 탐지"""
        print(f"\n=== 이상치 탐지 ({method.upper()} 방법) ===")
        
        numeric_cols = ['순방향유효전력', '지상무효', '진상무효', '피상전력']
        outliers_summary = {}
        
        for col in numeric_cols:
            data = self.lp_data[col].dropna()
            
            if method == 'iqr':
                Q1 = data.quantile(0.25)
                Q3 = data.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                outliers = data[(data < lower_bound) | (data > upper_bound)]
                
            elif method == 'zscore':
                z_scores = np.abs((data - data.mean()) / data.std())
                outliers = data[z_scores > 3]
            
            outliers_summary[col] = {
                'count': len(outliers),
                'percentage': len(outliers) / len(data) * 100,
                'min_outlier': outliers.min() if len(outliers) > 0 else None,
                'max_outlier': outliers.max() if len(outliers) > 0 else None
            }
            
            print(f"\n📊 {col}:")
            print(f"  이상치: {len(outliers):,}건 ({len(outliers)/len(data)*100:.2f}%)")
            if len(outliers) > 0:
                print(f"  범위: {outliers.min():.1f} ~ {outliers.max():.1f}")
        
        return outliers_summary
    
    def generate_quality_report(self):
        """데이터 품질 종합 리포트 생성"""
        print("\n" + "="*60)
        print("📋 데이터 품질 종합 리포트")
        print("="*60)
        
        # 1. 데이터 규모
        print("\n🔢 데이터 규모:")
        print(f"  • 고객 기본정보: {len(self.customer_data):,}명")
        print(f"  • LP 데이터: {len(self.lp_data):,}레코드")
        print(f"  • 분석 대상 고객: {self.lp_data['대체고객번호'].nunique()}명")
        print(f"  • 분석 기간: {self.lp_data['LP수신일자'].min()} ~ {self.lp_data['LP수신일자'].max()}")
        
        # 2. 데이터 품질 요약
        print("\n✅ 데이터 품질 상태:")
        numeric_cols = ['순방향유효전력', '지상무효', '진상무효', '피상전력']
        
        total_records = len(self.lp_data)
        total_nulls = sum(self.lp_data[col].isnull().sum() for col in numeric_cols)
        total_zeros = sum((self.lp_data[col] == 0).sum() for col in numeric_cols)
        total_negatives = sum((self.lp_data[col] < 0).sum() for col in numeric_cols)
        
        print(f"  • 결측치 비율: {total_nulls/(total_records*4)*100:.2f}%")
        print(f"  • 0값 비율: {total_zeros/(total_records*4)*100:.2f}%") 
        print(f"  • 음수값 비율: {total_negatives/(total_records*4)*100:.2f}%")
        
        # 3. 권장사항
        print("\n💡 다음 단계 권장사항:")
        print("  1. 시계열 패턴 분석 (일/주/월별 사용 패턴)")
        print("  2. 고객별 사용량 프로파일링")
        print("  3. 변동성 지표 계산 및 비교")
        print("  4. 이상 패턴 탐지 알고리즘 개발")
        
        return True

# 사용 예제
if __name__ == "__main__":
    # 분석기 초기화
    analyzer = KEPCODataAnalyzer()
    
    # 1단계: 고객 기본정보 분석
    customer_analysis = analyzer.load_customer_data('고객번호.xlsx')
    
    # 2단계: LP 데이터 분석  
    lp_analysis = analyzer.load_lp_data(['LP데이터1.csv', 'LP데이터2.csv'])
    
    # 3단계: 이상치 탐지
    outliers = analyzer.detect_outliers('iqr')
    
    # 4단계: 종합 리포트
    analyzer.generate_quality_report()
    
    print("\n🎯 1단계 데이터 품질 점검 완료!")
    print("다음: 2단계 시계열 패턴 분석 준비 완료")

=== 고객 기본정보 로딩 ===
총 고객 수: 200명
컬럼: ['순번', '고객번호', '계약전력', '계약종별', '사용용도', '주생산품', '산업분류']

기본 정보:
   순번   고객번호     계약전력            계약종별     사용용도  주생산품     산업분류
0   1  A1001  100~199  226 일반용(을) 고압A   02 상업용   제조업  299기타업종
1   2  A1002  500~599  222 일반용(갑)‖고압A   02 상업용    교회  769기타업종
2   3  A1003  700~799  322 산업용(갑)‖고압A  09 광공업용  금속가공  858기타업종
3   4  A1004  800~899  322 산업용(갑)‖고압A   02 상업용    상가  401기타업종
4   5  A1005  800~899  226 일반용(을) 고압A   02 상업용   제조업  364기타업종

=== 고객 분포 분석 ===

📊 계약종별 분포:
  322 산업용(갑)‖고압A: 56명 (28.0%)
  222 일반용(갑)‖고압A: 52명 (26.0%)
  226 일반용(을) 고압A: 46명 (23.0%)
  726 산업용(을) 고압A: 46명 (23.0%)

🏭 사용용도별 분포:
  02 상업용: 101명 (50.5%)
  09 광공업용: 99명 (49.5%)

⚡ 계약전력 분포:
  100~199kW: 28명 (14.0%)
  200~299kW: 30명 (15.0%)
  400~499kW: 31명 (15.5%)
  500~599kW: 38명 (19.0%)
  700~799kW: 27명 (13.5%)
  800~899kW: 46명 (23.0%)

=== LP 데이터 로딩 ===
파일 1 로딩: LP데이터1.csv
  레코드 수: 14,400
  고객 수: 10
  기간: 2024-03-01-00:00 ~ 2024-03-15-23:45
파일 2 로딩: LP데이터2.csv
  레코드 수: 15,360
  고객 수: 10
  기

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class KEPCOTimeSeriesAnalyzer:
    def __init__(self):
        self.lp_data = None
        
    def load_sample_data(self):
        """샘플 LP 데이터 로딩 및 전처리"""
        print("=== LP 데이터 로딩 및 전처리 ===")
        
        # 실제 환경에서는 CSV 파일 읽기
        # lp1 = pd.read_csv('LP데이터1.csv')
        # lp2 = pd.read_csv('LP데이터2.csv') 
        # self.lp_data = pd.concat([lp1, lp2], ignore_index=True)
        
        # 테스트용 샘플 데이터 생성
        self.lp_data = self._create_comprehensive_sample_data()
        
        # 날짜/시간 전처리
        self.lp_data['datetime'] = pd.to_datetime(self.lp_data['LP수신일자'], format='%Y-%m-%d-%H:%M')
        self.lp_data['date'] = self.lp_data['datetime'].dt.date
        self.lp_data['hour'] = self.lp_data['datetime'].dt.hour
        self.lp_data['minute'] = self.lp_data['datetime'].dt.minute
        self.lp_data['weekday'] = self.lp_data['datetime'].dt.weekday  # 0=월요일
        self.lp_data['is_weekend'] = self.lp_data['weekday'].isin([5, 6])
        
        print(f"✅ 데이터 로딩 완료: {len(self.lp_data):,}레코드")
        print(f"기간: {self.lp_data['datetime'].min()} ~ {self.lp_data['datetime'].max()}")
        print(f"고객 수: {self.lp_data['대체고객번호'].nunique()}명")
        
        return self.lp_data
    
    def _create_comprehensive_sample_data(self):
        """포괄적인 테스트용 샘플 데이터 생성"""
        data = []
        customers = [f'A{1001+i}' for i in range(10)]  # A1001~A1010
        start_date = datetime(2024, 3, 1)
        days = 31  # 3월 전체
        
        # 고객별 특성 정의 (다양한 패턴)
        customer_profiles = {
            'A1001': {'type': 'hospital', 'base_power': 120, 'peak_hours': [9, 14, 20], 'weekend_factor': 0.8},
            'A1002': {'type': 'office', 'base_power': 80, 'peak_hours': [9, 14], 'weekend_factor': 0.3},
            'A1003': {'type': 'retail', 'base_power': 100, 'peak_hours': [11, 15, 19], 'weekend_factor': 1.2},
            'A1004': {'type': 'factory', 'base_power': 150, 'peak_hours': [8, 13, 18], 'weekend_factor': 0.1},
            'A1005': {'type': 'restaurant', 'base_power': 90, 'peak_hours': [12, 18], 'weekend_factor': 1.1},
            'A1006': {'type': 'gym', 'base_power': 70, 'peak_hours': [7, 18, 21], 'weekend_factor': 1.3},
            'A1007': {'type': 'school', 'base_power': 110, 'peak_hours': [10, 14], 'weekend_factor': 0.2},
            'A1008': {'type': 'hotel', 'base_power': 130, 'peak_hours': [8, 20], 'weekend_factor': 1.0},
            'A1009': {'type': 'warehouse', 'base_power': 60, 'peak_hours': [9, 16], 'weekend_factor': 0.5},
            'A1010': {'type': 'clinic', 'base_power': 85, 'peak_hours': [10, 15], 'weekend_factor': 0.6}
        }
        
        for customer in customers:
            profile = customer_profiles[customer]
            
            for day in range(days):
                current_date = start_date + timedelta(days=day)
                is_weekend = current_date.weekday() >= 5
                
                for hour in range(24):
                    for minute in [0, 15, 30, 45]:
                        timestamp = current_date.replace(hour=hour, minute=minute)
                        
                        # 기본 전력 계산
                        base_power = profile['base_power']
                        
                        # 시간대별 패턴 (사인파 기반)
                        time_factor = 0.3 + 0.7 * (np.sin(2 * np.pi * hour / 24) + 1) / 2
                        
                        # 피크 시간 보정
                        peak_factor = 1.0
                        for peak_hour in profile['peak_hours']:
                            if abs(hour - peak_hour) <= 1:
                                peak_factor = 1.5
                        
                        # 주말 보정
                        weekend_factor = profile['weekend_factor'] if is_weekend else 1.0
                        
                        # 최종 전력 계산
                        power = base_power * time_factor * peak_factor * weekend_factor
                        power += np.random.normal(0, power * 0.1)  # 10% 노이즈
                        power = max(0, power)
                        
                        # 무효전력 계산
                        reactive_lag = power * np.random.uniform(0.1, 0.3)
                        reactive_lead = power * np.random.uniform(0.05, 0.15)
                        apparent_power = np.sqrt(power**2 + (reactive_lag - reactive_lead)**2)
                        
                        data.append({
                            '대체고객번호': customer,
                            'LP수신일자': timestamp.strftime('%Y-%m-%d-%H:%M'),
                            '순방향유효전력': round(power, 1),
                            '지상무효': round(reactive_lag, 1),
                            '진상무효': round(reactive_lead, 1),
                            '피상전력': round(apparent_power, 1)
                        })
        
        return pd.DataFrame(data)
    
    def analyze_hourly_patterns(self):
        """시간대별 전력 사용 패턴 분석"""
        print("\n=== 시간대별 전력 사용 패턴 분석 ===")
        
        # 시간대별 평균 사용량
        hourly_avg = self.lp_data.groupby('hour')['순방향유효전력'].agg(['mean', 'std', 'min', 'max']).round(1)
        
        print("📊 시간대별 평균 전력 사용량 (kW):")
        print("시간\t평균\t표준편차\t최소\t최대")
        for hour in range(24):
            stats = hourly_avg.loc[hour]
            print(f"{hour:02d}시\t{stats['mean']}\t{stats['std']}\t{stats['min']}\t{stats['max']}")
        
        # 피크/비피크 시간대 식별
        peak_threshold = hourly_avg['mean'].quantile(0.8)
        peak_hours = hourly_avg[hourly_avg['mean'] >= peak_threshold].index.tolist()
        off_peak_hours = hourly_avg[hourly_avg['mean'] < hourly_avg['mean'].quantile(0.3)].index.tolist()
        
        print(f"\n⚡ 피크 시간대 (상위 20%): {peak_hours}시")
        print(f"💤 비피크 시간대 (하위 30%): {off_peak_hours}시")
        
        return {
            'hourly_stats': hourly_avg,
            'peak_hours': peak_hours,
            'off_peak_hours': off_peak_hours
        }
    
    def analyze_daily_patterns(self):
        """일별/요일별 패턴 분석"""
        print("\n=== 일별/요일별 패턴 분석 ===")
        
        # 일별 총 사용량
        daily_usage = self.lp_data.groupby(['대체고객번호', 'date'])['순방향유효전력'].sum().reset_index()
        daily_usage['weekday'] = pd.to_datetime(daily_usage['date']).dt.weekday
        daily_usage['is_weekend'] = daily_usage['weekday'].isin([5, 6])
        
        # 요일별 평균 사용량
        weekday_avg = daily_usage.groupby('weekday')['순방향유효전력'].agg(['mean', 'std']).round(1)
        weekday_names = ['월', '화', '수', '목', '금', '토', '일']
        
        print("📅 요일별 평균 일간 사용량 (kWh):")
        for i, day_name in enumerate(weekday_names):
            stats = weekday_avg.loc[i]
            print(f"{day_name}요일: {stats['mean']:,.1f} ± {stats['std']:.1f}")
        
        # 평일 vs 주말 비교
        weekday_mean = daily_usage[~daily_usage['is_weekend']]['순방향유효전력'].mean()
        weekend_mean = daily_usage[daily_usage['is_weekend']]['순방향유효전력'].mean()
        weekend_ratio = weekend_mean / weekday_mean
        
        print(f"\n📊 평일 vs 주말 비교:")
        print(f"평일 평균: {weekday_mean:,.1f} kWh")
        print(f"주말 평균: {weekend_mean:,.1f} kWh")
        print(f"주말/평일 비율: {weekend_ratio:.2f}")
        
        return {
            'daily_usage': daily_usage,
            'weekday_stats': weekday_avg,
            'weekend_ratio': weekend_ratio
        }
    
    def analyze_customer_profiles(self):
        """고객별 사용량 프로파일 분석"""
        print("\n=== 고객별 사용량 프로파일 분석 ===")
        
        # 고객별 기본 통계
        customer_stats = self.lp_data.groupby('대체고객번호')['순방향유효전력'].agg([
            'count', 'mean', 'std', 'min', 'max'
        ]).round(1)
        customer_stats['cv'] = (customer_stats['std'] / customer_stats['mean']).round(3)  # 변동계수
        
        print("👥 고객별 기본 통계 (kW):")
        print("고객번호\t평균\t표준편차\t변동계수\t최소\t최대")
        for customer in customer_stats.index:
            stats = customer_stats.loc[customer]
            print(f"{customer}\t{stats['mean']}\t{stats['std']}\t{stats['cv']}\t{stats['min']}\t{stats['max']}")
        
        # 사용량 규모별 분류
        mean_usage = customer_stats['mean']
        high_users = mean_usage[mean_usage >= mean_usage.quantile(0.8)].index.tolist()
        low_users = mean_usage[mean_usage <= mean_usage.quantile(0.2)].index.tolist()
        
        print(f"\n📈 대용량 사용자 (상위 20%): {high_users}")
        print(f"📉 소용량 사용자 (하위 20%): {low_users}")
        
        # 변동성별 분류
        high_volatility = customer_stats[customer_stats['cv'] >= customer_stats['cv'].quantile(0.8)].index.tolist()
        low_volatility = customer_stats[customer_stats['cv'] <= customer_stats['cv'].quantile(0.2)].index.tolist()
        
        print(f"\n🌊 고변동성 고객: {high_volatility}")
        print(f"📊 저변동성 고객: {low_volatility}")
        
        return {
            'customer_stats': customer_stats,
            'usage_segments': {
                'high_users': high_users,
                'low_users': low_users,
                'high_volatility': high_volatility,
                'low_volatility': low_volatility
            }
        }
    
    def calculate_load_factors(self):
        """부하율 및 효율성 지표 계산"""
        print("\n=== 부하율 및 효율성 지표 계산 ===")
        
        # 고객별 부하율 계산
        customer_load_factors = {}
        
        for customer in self.lp_data['대체고객번호'].unique():
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer]
            
            avg_load = customer_data['순방향유효전력'].mean()
            max_load = customer_data['순방향유효전력'].max()
            load_factor = avg_load / max_load if max_load > 0 else 0
            
            # 피크 집중도 (피크 시간대 사용량 비중)
            peak_hours = [9, 14, 18]  # 대표 피크 시간
            peak_usage = customer_data[customer_data['hour'].isin(peak_hours)]['순방향유효전력'].mean()
            total_avg = customer_data['순방향유효전력'].mean()
            peak_concentration = peak_usage / total_avg if total_avg > 0 else 0
            
            customer_load_factors[customer] = {
                'load_factor': round(load_factor, 3),
                'peak_concentration': round(peak_concentration, 3),
                'avg_load': round(avg_load, 1),
                'max_load': round(max_load, 1)
            }
        
        print("⚡ 고객별 부하율 및 피크 집중도:")
        print("고객번호\t부하율\t피크집중도\t평균부하\t최대부하")
        for customer, metrics in customer_load_factors.items():
            print(f"{customer}\t{metrics['load_factor']}\t{metrics['peak_concentration']}\t{metrics['avg_load']}\t{metrics['max_load']}")
        
        # 전체 부하율 분포
        load_factors = [metrics['load_factor'] for metrics in customer_load_factors.values()]
        avg_load_factor = np.mean(load_factors)
        
        print(f"\n📊 전체 부하율 분포:")
        print(f"평균 부하율: {avg_load_factor:.3f}")
        print(f"부하율 범위: {min(load_factors):.3f} ~ {max(load_factors):.3f}")
        
        return customer_load_factors
    
    def detect_usage_anomalies(self):
        """사용량 이상 패턴 탐지"""
        print("\n=== 사용량 이상 패턴 탐지 ===")
        
        anomalies = []
        
        for customer in self.lp_data['대체고객번호'].unique():
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer].copy()
            customer_data = customer_data.sort_values('datetime')
            
            # 1. 급격한 변화 탐지 (전시점 대비 200% 이상 변화)
            customer_data['power_change'] = customer_data['순방향유효전력'].pct_change()
            sudden_changes = customer_data[abs(customer_data['power_change']) > 2.0]
            
            # 2. 연속적인 0값 탐지 (2시간 이상)
            customer_data['is_zero'] = customer_data['순방향유효전력'] == 0
            customer_data['zero_group'] = (customer_data['is_zero'] != customer_data['is_zero'].shift()).cumsum()
            zero_periods = customer_data[customer_data['is_zero']].groupby('zero_group').size()
            long_zero_periods = zero_periods[zero_periods >= 8]  # 2시간 = 8개 15분 구간
            
            # 3. 통계적 이상치 (Z-score > 3)
            mean_power = customer_data['순방향유효전력'].mean()
            std_power = customer_data['순방향유효전력'].std()
            if std_power > 0:
                customer_data['z_score'] = abs(customer_data['순방향유효전력'] - mean_power) / std_power
                statistical_outliers = customer_data[customer_data['z_score'] > 3]
            else:
                statistical_outliers = pd.DataFrame()
            
            # 이상치 정보 저장
            if len(sudden_changes) > 0 or len(long_zero_periods) > 0 or len(statistical_outliers) > 0:
                anomalies.append({
                    'customer': customer,
                    'sudden_changes': len(sudden_changes),
                    'long_zero_periods': len(long_zero_periods),
                    'statistical_outliers': len(statistical_outliers)
                })
        
        print("🚨 이상 패턴 탐지 결과:")
        if anomalies:
            print("고객번호\t급격변화\t장기0값\t통계이상치")
            for anomaly in anomalies:
                print(f"{anomaly['customer']}\t{anomaly['sudden_changes']}\t{anomaly['long_zero_periods']}\t{anomaly['statistical_outliers']}")
        else:
            print("✅ 심각한 이상 패턴 없음")
        
        return anomalies
    
    def generate_pattern_summary(self):
        """패턴 분석 종합 요약"""
        print("\n" + "="*60)
        print("📊 시계열 패턴 분석 종합 요약")
        print("="*60)
        
        # 주요 패턴 특성
        hourly_stats = self.analyze_hourly_patterns()
        daily_stats = self.analyze_daily_patterns()
        customer_stats = self.analyze_customer_profiles()
        load_factors = self.calculate_load_factors()
        anomalies = self.detect_usage_anomalies()
        
        print("\n🔍 주요 발견사항:")
        
        # 1. 시간 패턴
        peak_hours = hourly_stats['peak_hours']
        print(f"  • 주요 피크 시간: {peak_hours}시")
        
        # 2. 요일 패턴  
        weekend_ratio = daily_stats['weekend_ratio']
        print(f"  • 주말/평일 사용량 비율: {weekend_ratio:.2f}")
        
        # 3. 고객 다양성
        cv_range = customer_stats['customer_stats']['cv']
        print(f"  • 고객별 변동계수 범위: {cv_range.min():.3f} ~ {cv_range.max():.3f}")
        
        # 4. 부하율
        load_factor_avg = np.mean([lf['load_factor'] for lf in load_factors.values()])
        print(f"  • 평균 부하율: {load_factor_avg:.3f}")
        
        # 5. 이상 패턴
        anomaly_customers = len(anomalies)
        print(f"  • 이상 패턴 고객: {anomaly_customers}명")
        
        print("\n💡 변동계수 설계를 위한 인사이트:")
        print("  1. 시간대별 가중치 필요 (피크/비피크 구분)")
        print("  2. 요일별 보정 계수 고려") 
        print("  3. 고객별 기준 변동성 설정")
        print("  4. 부하율과 변동성의 상관관계 분석")
        print("  5. 다차원 변동성 지표 조합 검토")
        
        return {
            'hourly_patterns': hourly_stats,
            'daily_patterns': daily_stats,
            'customer_profiles': customer_stats,
            'load_factors': load_factors,
            'anomalies': anomalies
        }

# 사용 예제
if __name__ == "__main__":
    # 분석기 초기화
    analyzer = KEPCOTimeSeriesAnalyzer()
    
    # 데이터 로딩
    lp_data = analyzer.load_sample_data()
    
    # 패턴 분석 실행
    pattern_summary = analyzer.generate_pattern_summary()
    
    print("\n🎯 2단계 시계열 패턴 분석 완료!")
    print("다음: 3단계 변동성 지표 계산")

=== LP 데이터 로딩 및 전처리 ===
✅ 데이터 로딩 완료: 29,760레코드
기간: 2024-03-01 00:00:00 ~ 2024-03-31 23:45:00
고객 수: 10명

📊 시계열 패턴 분석 종합 요약

=== 시간대별 전력 사용 패턴 분석 ===
📊 시간대별 평균 전력 사용량 (kW):
시간	평균	표준편차	최소	최대
00시	57.9	23.7	7.4	117.3
01시	65.9	27.1	9.0	136.7
02시	73.3	30.1	10.8	149.4
03시	80.0	33.2	11.5	175.2
04시	84.8	34.7	12.1	171.1
05시	87.9	36.3	12.0	179.3
06시	92.8	37.4	12.9	173.8
07시	103.9	55.6	16.9	267.3
08시	110.7	53.5	15.3	256.7
09시	107.8	51.3	16.4	255.3
10시	94.3	38.5	10.8	177.4
11시	79.3	34.8	8.6	157.4
12시	67.8	34.6	11.1	180.9
13시	62.6	30.7	10.0	162.1
14시	55.0	26.4	8.4	138.2
15시	45.6	18.6	5.1	88.3
16시	35.0	14.3	3.9	77.4
17시	32.8	14.8	4.8	87.4
18시	32.5	16.0	4.8	89.2
19시	37.5	19.2	5.5	92.0
20시	38.2	19.0	4.0	88.8
21시	42.4	20.9	4.6	94.4
22시	44.3	17.9	5.2	86.1
23시	50.0	20.7	6.4	101.0

⚡ 피크 시간대 (상위 20%): [6, 7, 8, 9, 10]시
💤 비피크 시간대 (하위 30%): [16, 17, 18, 19, 20, 21, 22]시

=== 일별/요일별 패턴 분석 ===
📅 요일별 평균 일간 사용량 (kWh):
월요일: 7,063.7 ± 1993.7
화요일: 7,074.2 ± 1988.2
수요일: 7,053.0 ± 1999.1
목요일: 7,054.8 ± 2016.7
금요일: 7,03

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

class KEPCOVolatilityAnalyzer:
    def __init__(self):
        self.lp_data = None
        self.volatility_metrics = {}
        
    def load_and_prepare_data(self):
        """데이터 로딩 및 전처리"""
        print("=== 변동성 분석을 위한 데이터 준비 ===")
        
        # 실제 환경에서는 CSV 파일 읽기
        # self.lp_data = pd.concat([
        #     pd.read_csv('LP데이터1.csv'),
        #     pd.read_csv('LP데이터2.csv')
        # ], ignore_index=True)
        
        # 테스트용 데이터 생성
        self.lp_data = self._create_volatility_test_data()
        
        # 시간 관련 컬럼 추가
        self.lp_data['datetime'] = pd.to_datetime(self.lp_data['LP수신일자'], format='%Y-%m-%d-%H:%M')
        self.lp_data['date'] = self.lp_data['datetime'].dt.date
        self.lp_data['hour'] = self.lp_data['datetime'].dt.hour
        self.lp_data['weekday'] = self.lp_data['datetime'].dt.weekday
        
        print(f"✅ 데이터 준비 완료: {len(self.lp_data):,}레코드")
        return self.lp_data
    
    def _create_volatility_test_data(self):
        """변동성 테스트용 데이터 생성"""
        data = []
        customers = [f'A{1001+i}' for i in range(10)]
        start_date = datetime(2024, 3, 1)
        days = 31
        
        # 다양한 변동성 패턴을 가진 고객 정의
        volatility_profiles = {
            'A1001': {'type': 'stable', 'base': 100, 'noise': 0.05},      # 안정적
            'A1002': {'type': 'high_volatility', 'base': 80, 'noise': 0.3}, # 고변동성
            'A1003': {'type': 'periodic', 'base': 120, 'noise': 0.1},     # 주기적
            'A1004': {'type': 'trending', 'base': 90, 'noise': 0.15},     # 트렌드
            'A1005': {'type': 'seasonal', 'base': 110, 'noise': 0.2},     # 계절성
            'A1006': {'type': 'jumpy', 'base': 85, 'noise': 0.1},         # 점프형
            'A1007': {'type': 'clustered', 'base': 95, 'noise': 0.25},    # 클러스터형
            'A1008': {'type': 'low_usage', 'base': 30, 'noise': 0.4},     # 저사용량
            'A1009': {'type': 'medium', 'base': 70, 'noise': 0.18},       # 중간
            'A1010': {'type': 'irregular', 'base': 105, 'noise': 0.35}    # 불규칙
        }
        
        for customer in customers:
            profile = volatility_profiles[customer]
            
            for day in range(days):
                current_date = start_date + timedelta(days=day)
                
                for hour in range(24):
                    for minute in [0, 15, 30, 45]:
                        timestamp = current_date.replace(hour=hour, minute=minute)
                        
                        # 기본 패턴 생성
                        base_power = profile['base']
                        
                        # 타입별 패턴 적용
                        if profile['type'] == 'stable':
                            power = base_power + np.random.normal(0, base_power * profile['noise'])
                        
                        elif profile['type'] == 'high_volatility':
                            # 높은 변동성 - 큰 무작위 변화
                            power = base_power + np.random.normal(0, base_power * profile['noise'])
                            if np.random.random() < 0.1:  # 10% 확률로 큰 점프
                                power *= np.random.choice([0.3, 2.5])
                        
                        elif profile['type'] == 'periodic':
                            # 주기적 패턴
                            daily_cycle = np.sin(2 * np.pi * hour / 24)
                            weekly_cycle = np.sin(2 * np.pi * day / 7)
                            power = base_power * (1 + 0.3 * daily_cycle + 0.1 * weekly_cycle)
                            power += np.random.normal(0, power * profile['noise'])
                        
                        elif profile['type'] == 'trending':
                            # 트렌드 패턴
                            trend = 0.5 * day / days  # 30일간 50% 증가
                            power = base_power * (1 + trend)
                            power += np.random.normal(0, power * profile['noise'])
                        
                        elif profile['type'] == 'seasonal':
                            # 계절성 패턴
                            seasonal = 0.2 * np.sin(2 * np.pi * day / 30)
                            power = base_power * (1 + seasonal)
                            power += np.random.normal(0, power * profile['noise'])
                        
                        elif profile['type'] == 'jumpy':
                            # 급격한 변화가 있는 패턴
                            power = base_power
                            if day % 7 == 0 and hour == 9:  # 주 1회 큰 변화
                                power *= 2.0
                            elif day % 7 == 3 and hour == 15:
                                power *= 0.4
                            power += np.random.normal(0, power * profile['noise'])
                        
                        elif profile['type'] == 'clustered':
                            # 변동성 클러스터링 (변동성이 높은 구간과 낮은 구간)
                            if day % 10 < 3:  # 30% 기간 동안 높은 변동성
                                noise_factor = profile['noise'] * 2
                            else:
                                noise_factor = profile['noise'] * 0.5
                            power = base_power + np.random.normal(0, base_power * noise_factor)
                        
                        else:  # irregular, low_usage, medium
                            power = base_power + np.random.normal(0, base_power * profile['noise'])
                        
                        power = max(0, power)  # 음수 방지
                        
                        data.append({
                            '대체고객번호': customer,
                            'LP수신일자': timestamp.strftime('%Y-%m-%d-%H:%M'),
                            '순방향유효전력': round(power, 1),
                            '지상무효': round(power * np.random.uniform(0.1, 0.3), 1),
                            '진상무효': round(power * np.random.uniform(0.05, 0.15), 1),
                            '피상전력': round(power * np.random.uniform(1.0, 1.1), 1)
                        })
        
        return pd.DataFrame(data)
    
    def calculate_basic_volatility_metrics(self):
        """기본 변동성 지표 계산"""
        print("\n=== 기본 변동성 지표 계산 ===")
        
        metrics = {}
        
        for customer in self.lp_data['대체고객번호'].unique():
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer]['순방향유효전력']
            
            # 1. 전통적 변동계수 (CV)
            cv = customer_data.std() / customer_data.mean() if customer_data.mean() > 0 else 0
            
            # 2. 범위 기반 변동성
            range_volatility = (customer_data.max() - customer_data.min()) / customer_data.mean() if customer_data.mean() > 0 else 0
            
            # 3. 분위수 기반 변동성 (IQR/Median)
            q75, q25 = np.percentile(customer_data, [75, 25])
            iqr_volatility = (q75 - q25) / np.median(customer_data) if np.median(customer_data) > 0 else 0
            
            # 4. 평균절대편차 (MAD)
            mad = np.mean(np.abs(customer_data - customer_data.mean()))
            mad_volatility = mad / customer_data.mean() if customer_data.mean() > 0 else 0
            
            # 5. 변화율 기반 변동성
            returns = customer_data.pct_change().dropna()
            return_volatility = returns.std() if len(returns) > 0 else 0
            
            metrics[customer] = {
                'cv': round(cv, 4),
                'range_vol': round(range_volatility, 4),
                'iqr_vol': round(iqr_volatility, 4),
                'mad_vol': round(mad_volatility, 4),
                'return_vol': round(return_volatility, 4)
            }
        
        print("고객번호\tCV\t범위변동성\tIQR변동성\tMAD변동성\t수익률변동성")
        for customer, metrics_dict in metrics.items():
            print(f"{customer}\t{metrics_dict['cv']}\t{metrics_dict['range_vol']}\t{metrics_dict['iqr_vol']}\t{metrics_dict['mad_vol']}\t{metrics_dict['return_vol']}")
        
        self.volatility_metrics['basic'] = metrics
        return metrics
    
    def calculate_time_window_volatility(self):
        """시간 윈도우별 변동성 계산"""
        print("\n=== 시간 윈도우별 변동성 분석 ===")
        
        window_metrics = {}
        windows = {
            'hourly': 4,    # 1시간 (4개 15분 구간)
            'daily': 96,    # 1일 (96개 15분 구간)
            'weekly': 672   # 1주 (672개 15분 구간)
        }
        
        for customer in self.lp_data['대체고객번호'].unique():
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer].sort_values('datetime')
            power_series = customer_data['순방향유효전력']
            
            window_metrics[customer] = {}
            
            for window_name, window_size in windows.items():
                # 롤링 윈도우로 변동계수 계산
                rolling_std = power_series.rolling(window=window_size, min_periods=window_size//2).std()
                rolling_mean = power_series.rolling(window=window_size, min_periods=window_size//2).mean()
                rolling_cv = rolling_std / rolling_mean
                
                # 윈도우별 변동성 통계
                window_metrics[customer][window_name] = {
                    'mean_cv': round(rolling_cv.mean(), 4),
                    'std_cv': round(rolling_cv.std(), 4),
                    'max_cv': round(rolling_cv.max(), 4),
                    'min_cv': round(rolling_cv.min(), 4)
                }
        
        print("윈도우별 평균 변동계수:")
        print("고객번호\t시간별\t일별\t주별")
        for customer in window_metrics.keys():
            hourly_cv = window_metrics[customer]['hourly']['mean_cv']
            daily_cv = window_metrics[customer]['daily']['mean_cv']
            weekly_cv = window_metrics[customer]['weekly']['mean_cv']
            print(f"{customer}\t{hourly_cv}\t{daily_cv}\t{weekly_cv}")
        
        self.volatility_metrics['time_windows'] = window_metrics
        return window_metrics
    
    def calculate_directional_volatility(self):
        """방향성 변동성 분석 (상승/하락 비대칭성)"""
        print("\n=== 방향성 변동성 분석 ===")
        
        directional_metrics = {}
        
        for customer in self.lp_data['대체고객번호'].unique():
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer].sort_values('datetime')
            power_series = customer_data['순방향유효전력']
            
            # 변화율 계산
            returns = power_series.pct_change().dropna()
            
            # 상승/하락 분리
            upside_returns = returns[returns > 0]
            downside_returns = returns[returns < 0]
            
            # 방향별 변동성
            upside_volatility = upside_returns.std() if len(upside_returns) > 0 else 0
            downside_volatility = abs(downside_returns.std()) if len(downside_returns) > 0 else 0
            
            # 비대칭성 지수
            asymmetry_ratio = upside_volatility / downside_volatility if downside_volatility > 0 else 0
            
            # 급격한 변화 횟수
            large_increases = len(returns[returns > 0.5])  # 50% 이상 증가
            large_decreases = len(returns[returns < -0.5]) # 50% 이상 감소
            
            directional_metrics[customer] = {
                'upside_vol': round(upside_volatility, 4),
                'downside_vol': round(downside_volatility, 4),
                'asymmetry_ratio': round(asymmetry_ratio, 4),
                'large_increases': large_increases,
                'large_decreases': large_decreases
            }
        
        print("고객번호\t상승변동성\t하락변동성\t비대칭비율\t급증횟수\t급감횟수")
        for customer, metrics_dict in directional_metrics.items():
            print(f"{customer}\t{metrics_dict['upside_vol']}\t{metrics_dict['downside_vol']}\t{metrics_dict['asymmetry_ratio']}\t{metrics_dict['large_increases']}\t{metrics_dict['large_decreases']}")
        
        self.volatility_metrics['directional'] = directional_metrics
        return directional_metrics
    
    def calculate_pattern_stability(self):
        """패턴 안정성 분석"""
        print("\n=== 패턴 안정성 분석 ===")
        
        stability_metrics = {}
        
        for customer in self.lp_data['대체고객번호'].unique():
            customer_data = self.lp_data[self.lp_data['대체고객번호'] == customer]
            
            # 시간대별 패턴 일관성
            hourly_patterns = []
            for day in customer_data['date'].unique():
                day_data = customer_data[customer_data['date'] == day]
                hourly_avg = day_data.groupby('hour')['순방향유효전력'].mean()
                hourly_patterns.append(hourly_avg.values)
            
            # 패턴 간 상관관계 (일관성 측정)
            if len(hourly_patterns) > 1:
                correlations = []
                for i in range(len(hourly_patterns)):
                    for j in range(i+1, len(hourly_patterns)):
                        if len(hourly_patterns[i]) == len(hourly_patterns[j]):
                            corr = np.corrcoef(hourly_patterns[i], hourly_patterns[j])[0,1]
                            if not np.isnan(corr):
                                correlations.append(corr)
                
                pattern_consistency = np.mean(correlations) if correlations else 0
            else:
                pattern_consistency = 0
            
            # 주기성 강도 (FFT 기반)
            power_series = customer_data.sort_values('datetime')['순방향유효전력'].values
            if len(power_series) > 100:  # 충분한 데이터가 있을 때만
                fft = np.fft.fft(power_series)
                fft_magnitude = np.abs(fft)
                
                # 일일 주기 (96포인트) 강도
                daily_freq_idx = len(fft_magnitude) // 96 if len(fft_magnitude) >= 96 else 1
                daily_periodicity = fft_magnitude[daily_freq_idx] / np.mean(fft_magnitude) if np.mean(fft_magnitude) > 0 else 0
            else:
                daily_periodicity = 0
            
            # 예측가능성 (자기상관)
            autocorr_1lag = power_series[1:].dot(power_series[:-1]) / (np.linalg.norm(power_series[1:]) * np.linalg.norm(power_series[:-1])) if len(power_series) > 1 else 0
            
            stability_metrics[customer] = {
                'pattern_consistency': round(pattern_consistency, 4),
                'daily_periodicity': round(daily_periodicity, 4),
                'autocorrelation': round(autocorr_1lag, 4)
            }
        
        print("고객번호\t패턴일관성\t일일주기성\t자기상관")
        for customer, metrics_dict in stability_metrics.items():
            print(f"{customer}\t{metrics_dict['pattern_consistency']}\t{metrics_dict['daily_periodicity']}\t{metrics_dict['autocorrelation']}")
        
        self.volatility_metrics['stability'] = stability_metrics
        return stability_metrics
    
    def create_composite_volatility_score(self):
        """복합 변동성 스코어 생성"""
        print("\n=== 복합 변동성 스코어 계산 ===")
        
        if not all(key in self.volatility_metrics for key in ['basic', 'time_windows', 'directional', 'stability']):
            print("❌ 모든 변동성 지표를 먼저 계산해주세요.")
            return None
        
        composite_scores = {}
        
        for customer in self.lp_data['대체고객번호'].unique():
            # 각 카테고리별 점수 계산 (0-1 정규화)
            basic_score = self.volatility_metrics['basic'][customer]['cv']
            window_score = np.mean([
                self.volatility_metrics['time_windows'][customer]['hourly']['mean_cv'],
                self.volatility_metrics['time_windows'][customer]['daily']['mean_cv'],
                self.volatility_metrics['time_windows'][customer]['weekly']['mean_cv']
            ])
            directional_score = (
                self.volatility_metrics['directional'][customer]['upside_vol'] + 
                self.volatility_metrics['directional'][customer]['downside_vol']
            ) / 2
            
            # 안정성은 역수로 (낮은 안정성 = 높은 변동성)
            stability_score = 1 - self.volatility_metrics['stability'][customer]['pattern_consistency']
            
            # 가중 평균으로 최종 스코어 계산
            weights = {
                'basic': 0.3,       # 기본 변동계수
                'window': 0.3,      # 시간 윈도우 변동성
                'directional': 0.2, # 방향성 변동성
                'stability': 0.2    # 패턴 불안정성
            }
            
            composite_score = (
                weights['basic'] * basic_score +
                weights['window'] * window_score +
                weights['directional'] * directional_score +
                weights['stability'] * stability_score
            )
            
            composite_scores[customer] = {
                'basic_score': round(basic_score, 4),
                'window_score': round(window_score, 4),
                'directional_score': round(directional_score, 4),
                'stability_score': round(stability_score, 4),
                'composite_score': round(composite_score, 4)
            }
        
        print("고객번호\t기본\t윈도우\t방향성\t안정성\t복합점수")
        for customer, scores in composite_scores.items():
            print(f"{customer}\t{scores['basic_score']}\t{scores['window_score']}\t{scores['directional_score']}\t{scores['stability_score']}\t{scores['composite_score']}")
        
        # 변동성 등급 분류
        composite_values = [scores['composite_score'] for scores in composite_scores.values()]
        percentiles = np.percentile(composite_values, [33, 67])
        
        print(f"\n📊 변동성 등급 기준:")
        print(f"  저변동성: < {percentiles[0]:.3f}")
        print(f"  중변동성: {percentiles[0]:.3f} ~ {percentiles[1]:.3f}")
        print(f"  고변동성: > {percentiles[1]:.3f}")
        
        print("\n등급별 고객 분류:")
        for customer, scores in composite_scores.items():
            score = scores['composite_score']
            if score < percentiles[0]:
                grade = "저변동성"
            elif score < percentiles[1]:
                grade = "중변동성"
            else:
                grade = "고변동성"
            print(f"  {customer}: {score:.3f} ({grade})")
        
        self.volatility_metrics['composite'] = composite_scores
        return composite_scores
    
    def generate_volatility_report(self):
        """변동성 분석 종합 리포트"""
        print("\n" + "="*60)
        print("📊 변동성 분석 종합 리포트")
        print("="*60)
        
        # 모든 지표 계산
        basic_metrics = self.calculate_basic_volatility_metrics()
        window_metrics = self.calculate_time_window_volatility()
        directional_metrics = self.calculate_directional_volatility()
        stability_metrics = self.calculate_pattern_stability()
        composite_scores = self.create_composite_volatility_score()
        
        print("\n🎯 변동계수 알고리즘 설계 인사이트:")
        print("  1. 다차원 변동성 지표의 필요성 확인")
        print("  2. 시간 윈도우별 차별화된 가중치 적용")
        print("  3. 방향성 변동성으로 리스크 비대칭성 포착")
        print("  4. 패턴 안정성으로 예측가능성 평가")
        print("  5. 복합 스코어를 통한 종합적 변동성 평가")
        
        print("\n💡 스태킹 알고리즘 설계 방향:")
        print("  • Level-0 모델: 각 변동성 지표를 개별 모델로 구성")
        print("  • Level-1 메타모델: 가중 결합으로 최종 변동계수 산출")
        print("  • 과적합 방지: 교차검증 및 정규화 적용")
        
        return {
            'basic': basic_metrics,
            'time_windows': window_metrics,
            'directional': directional_metrics,
            'stability': stability_metrics,
            'composite': composite_scores
        }

# 사용 예제
if __name__ == "__main__":
    # 분석기 초기화
    analyzer = KEPCOVolatilityAnalyzer()
    
    # 데이터 로딩
    data = analyzer.load_and_prepare_data()
    
    # 종합 변동성 분석
    volatility_report = analyzer.generate_volatility_report()
    
    print("\n🎯 3단계 변동성 지표 계산 완료!")
    print("다음: 4단계 스태킹 알고리즘 개발")

=== 변동성 분석을 위한 데이터 준비 ===
✅ 데이터 준비 완료: 29,760레코드

📊 변동성 분석 종합 리포트

=== 기본 변동성 지표 계산 ===
고객번호	CV	범위변동성	IQR변동성	MAD변동성	수익률변동성
A1001	0.0506	0.3199	0.069	0.0404	0.0712
A1002	0.4685	3.819	0.4496	0.3145	1.132
A1003	0.246	1.2983	0.397	0.2091	0.151
A1004	0.1925	1.4063	0.2638	0.1541	0.2313
A1005	0.2487	1.6808	0.3404	0.1992	0.3534
A1006	0.1393	2.1185	0.1366	0.0893	0.1682
A1007	0.2909	2.5063	0.2315	0.1917	nan
A1008	0.3906	2.33	0.5331	0.313	nan
A1009	0.1806	1.309	0.2369	0.1441	0.2859
A1010	0.3419	2.1765	0.4417	0.2714	nan

=== 시간 윈도우별 변동성 분석 ===
윈도우별 평균 변동계수:
고객번호	시간별	일별	주별
A1001	0.0469	0.0507	0.051
A1002	0.3781	0.4642	0.4682
A1003	0.0964	0.2381	0.2449
A1004	0.1401	0.1515	0.1545
A1005	0.1923	0.2052	0.2138
A1006	0.0987	0.1282	0.1374
A1007	0.2277	0.2414	0.28
A1008	0.3766	0.3906	0.3891
A1009	0.1682	0.1808	0.181
A1010	0.3256	0.3406	0.3404

=== 방향성 변동성 분석 ===
고객번호	상승변동성	하락변동성	비대칭비율	급증횟수	급감횟수
A1001	0.0454	0.0392	1.1583	0	0
A1002	1.3262	0.2305	5.7542	667	379
A1003	0.1081	0.0758	1.4276	11	0
A1004	0.1816	0.1

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
warnings.filterwarnings('ignore')

class KEPCOVolatilityStackingModel:
    """
    한국전력공사 전력 사용패턴 변동계수 스태킹 알고리즘
    
    Level-0: 다양한 변동성 지표들을 개별 모델로 구성
    Level-1: 메타모델로 최종 변동계수 산출
    """
    
    def __init__(self):
        self.level0_models = {}
        self.level1_model = None
        self.scaler = None
        self.is_fitted = False
        
        # Level-0 모델 정의
        self._initialize_level0_models()
        
        # 과적합 방지 설정
        self.cv_folds = 5
        self.random_state = 42
        
    def _initialize_level0_models(self):
        """Level-0 기본 모델들 초기화"""
        self.level0_models = {
            'traditional_cv': self._traditional_cv_model,
            'range_volatility': self._range_volatility_model,
            'iqr_volatility': self._iqr_volatility_model,
            'mad_volatility': self._mad_volatility_model,
            'return_volatility': self._return_volatility_model,
            'window_volatility': self._window_volatility_model,
            'percentile_volatility': self._percentile_volatility_model
        }
        
    def _traditional_cv_model(self, data):
        """전통적 변동계수 모델"""
        return data.std() / data.mean() if data.mean() > 0 else 0
    
    def _range_volatility_model(self, data):
        """범위 기반 변동성 모델"""
        return (data.max() - data.min()) / data.mean() if data.mean() > 0 else 0
    
    def _iqr_volatility_model(self, data):
        """분위수 기반 변동성 모델"""
        q75, q25 = np.percentile(data, [75, 25])
        median = np.median(data)
        return (q75 - q25) / median if median > 0 else 0
    
    def _mad_volatility_model(self, data):
        """평균절대편차 기반 변동성 모델"""
        mad = np.mean(np.abs(data - data.mean()))
        return mad / data.mean() if data.mean() > 0 else 0
    
    def _return_volatility_model(self, data):
        """수익률 변동성 모델"""
        if len(data) < 2:
            return 0
        returns = data.pct_change().dropna()
        return returns.std() if len(returns) > 0 else 0
    
    def _window_volatility_model(self, data, window_size=96):
        """윈도우 기반 변동성 모델 (일별)"""
        if len(data) < window_size:
            return self._traditional_cv_model(data)
        
        rolling_cv = []
        for i in range(window_size, len(data) + 1):
            window_data = data.iloc[i-window_size:i]
            cv = self._traditional_cv_model(window_data)
            rolling_cv.append(cv)
        
        return np.mean(rolling_cv) if rolling_cv else 0
    
    def _percentile_volatility_model(self, data):
        """백분위수 기반 변동성 모델"""
        p90 = np.percentile(data, 90)
        p10 = np.percentile(data, 10)
        p50 = np.percentile(data, 50)
        return (p90 - p10) / p50 if p50 > 0 else 0
    
    def extract_features(self, lp_data):
        """
        LP 데이터에서 변동성 특성 추출
        
        Parameters:
        lp_data: DataFrame with columns ['대체고객번호', 'LP수신일자', '순방향유효전력', ...]
        
        Returns:
        features_df: DataFrame with volatility features for each customer
        """
        print("🔧 변동성 특성 추출 중...")
        
        features_list = []
        
        for customer in lp_data['대체고객번호'].unique():
            customer_data = lp_data[lp_data['대체고객번호'] == customer]
            power_data = customer_data['순방향유효전력']
            
            # 시간 정보 추가
            if 'datetime' not in customer_data.columns:
                customer_data = customer_data.copy()
                customer_data['datetime'] = pd.to_datetime(customer_data['LP수신일자'], format='%Y-%m-%d-%H:%M')
                customer_data['hour'] = customer_data['datetime'].dt.hour
                customer_data['weekday'] = customer_data['datetime'].dt.weekday
            
            # Level-0 모델들로 특성 계산
            features = {'customer_id': customer}
            
            for model_name, model_func in self.level0_models.items():
                try:
                    if model_name == 'window_volatility':
                        features[model_name] = model_func(power_data, window_size=96)
                    else:
                        features[model_name] = model_func(power_data)
                except:
                    features[model_name] = 0
            
            # 추가 컨텍스트 특성
            features.update({
                'mean_usage': power_data.mean(),
                'total_usage': power_data.sum(),
                'peak_ratio': power_data.max() / power_data.mean() if power_data.mean() > 0 else 0,
                'zero_ratio': (power_data == 0).sum() / len(power_data),
                'weekend_effect': self._calculate_weekend_effect(customer_data),
                'peak_hour_concentration': self._calculate_peak_concentration(customer_data)
            })
            
            features_list.append(features)
        
        features_df = pd.DataFrame(features_list)
        print(f"✅ 특성 추출 완료: {len(features_df)}명 고객, {len(features_df.columns)-1}개 특성")
        
        return features_df
    
    def _calculate_weekend_effect(self, customer_data):
        """주말 효과 계산"""
        try:
            weekday_avg = customer_data[customer_data['weekday'] < 5]['순방향유효전력'].mean()
            weekend_avg = customer_data[customer_data['weekday'] >= 5]['순방향유효전력'].mean()
            
            if weekday_avg > 0:
                return abs(weekend_avg / weekday_avg - 1)
            else:
                return 0
        except:
            return 0
    
    def _calculate_peak_concentration(self, customer_data):
        """피크 시간 집중도 계산"""
        try:
            peak_hours = [9, 14, 18]  # 일반적인 피크 시간
            peak_usage = customer_data[customer_data['hour'].isin(peak_hours)]['순방향유효전력'].mean()
            total_avg = customer_data['순방향유효전력'].mean()
            
            return peak_usage / total_avg if total_avg > 0 else 0
        except:
            return 0
    
    def prepare_training_data(self, features_df, target_column='traditional_cv'):
        """
        훈련 데이터 준비
        
        Parameters:
        features_df: 특성 데이터프레임
        target_column: 타겟 변수로 사용할 컬럼명
        
        Returns:
        X, y: 특성 행렬과 타겟 벡터
        """
        # 특성 선택 (Level-0 모델 출력들)
        feature_columns = list(self.level0_models.keys()) + [
            'mean_usage', 'peak_ratio', 'zero_ratio', 
            'weekend_effect', 'peak_hour_concentration'
        ]
        
        X = features_df[feature_columns].copy()
        y = features_df[target_column].copy()
        
        # 결측치 처리
        X = X.fillna(0)
        y = y.fillna(0)
        
        # 이상치 처리 (상위 5%, 하위 5% 클리핑)
        for col in X.columns:
            if X[col].dtype in ['float64', 'int64']:
                lower_bound = X[col].quantile(0.05)
                upper_bound = X[col].quantile(0.95)
                X[col] = X[col].clip(lower_bound, upper_bound)
        
        print(f"📊 훈련 데이터 준비 완료: {X.shape[0]}개 샘플, {X.shape[1]}개 특성")
        
        return X, y
    
    def fit(self, X, y):
        """
        스태킹 모델 훈련
        
        Parameters:
        X: 특성 행렬
        y: 타겟 벡터 (실제 변동계수)
        """
        print("🚀 스태킹 모델 훈련 시작...")
        
        # 데이터 정규화
        self.scaler = RobustScaler()  # 이상치에 강건한 스케일러
        X_scaled = self.scaler.fit_transform(X)
        
        # Level-1 메타모델 후보들
        meta_models = {
            'linear': LinearRegression(),
            'ridge': Ridge(alpha=1.0, random_state=self.random_state),
            'lasso': Lasso(alpha=0.1, random_state=self.random_state),
            'elastic': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=self.random_state),
            'rf': RandomForestRegressor(n_estimators=100, max_depth=5, random_state=self.random_state),
            'gbm': GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=self.random_state)
        }
        
        # 교차검증으로 최적 메타모델 선택
        best_score = -np.inf
        best_model = None
        best_model_name = None
        
        # 시계열 교차검증
        tscv = TimeSeriesSplit(n_splits=self.cv_folds)
        
        print("🔍 메타모델 성능 비교:")
        print("모델명\t\tMAE\t\tMSE\t\tR²")
        
        for model_name, model in meta_models.items():
            try:
                # 교차검증 점수 계산
                mae_scores = -cross_val_score(model, X_scaled, y, cv=tscv, scoring='neg_mean_absolute_error')
                mse_scores = -cross_val_score(model, X_scaled, y, cv=tscv, scoring='neg_mean_squared_error')
                r2_scores = cross_val_score(model, X_scaled, y, cv=tscv, scoring='r2')
                
                avg_mae = np.mean(mae_scores)
                avg_mse = np.mean(mse_scores)
                avg_r2 = np.mean(r2_scores)
                
                print(f"{model_name:<12}\t{avg_mae:.4f}\t\t{avg_mse:.4f}\t\t{avg_r2:.4f}")
                
                # R² 점수 기준으로 최적 모델 선택
                if avg_r2 > best_score:
                    best_score = avg_r2
                    best_model = model
                    best_model_name = model_name
                    
            except Exception as e:
                print(f"{model_name:<12}\t에러: {str(e)[:30]}...")
        
        # 최적 모델로 전체 데이터 훈련
        if best_model is not None:
            self.level1_model = best_model
            self.level1_model.fit(X_scaled, y)
            
            print(f"\n🏆 최적 메타모델: {best_model_name} (R² = {best_score:.4f})")
            
            # 특성 중요도 출력 (가능한 경우)
            if hasattr(self.level1_model, 'feature_importances_'):
                importance_df = pd.DataFrame({
                    'feature': X.columns,
                    'importance': self.level1_model.feature_importances_
                }).sort_values('importance', ascending=False)
                
                print("\n📊 특성 중요도 (상위 5개):")
                for _, row in importance_df.head().iterrows():
                    print(f"  {row['feature']}: {row['importance']:.4f}")
            
            elif hasattr(self.level1_model, 'coef_'):
                coef_df = pd.DataFrame({
                    'feature': X.columns,
                    'coefficient': self.level1_model.coef_
                }).sort_values('coefficient', key=abs, ascending=False)
                
                print("\n📊 모델 계수 (절댓값 상위 5개):")
                for _, row in coef_df.head().iterrows():
                    print(f"  {row['feature']}: {row['coefficient']:.4f}")
            
            self.is_fitted = True
            print("✅ 스태킹 모델 훈련 완료!")
            
        else:
            raise ValueError("❌ 적합한 메타모델을 찾을 수 없습니다.")
        
        return self
    
    def predict(self, features_df):
        """
        변동계수 예측
        
        Parameters:
        features_df: 특성 데이터프레임
        
        Returns:
        predictions: 예측된 변동계수
        """
        if not self.is_fitted:
            raise ValueError("❌ 모델이 훈련되지 않았습니다. fit() 메서드를 먼저 호출하세요.")
        
        # 훈련 시와 동일한 특성 선택
        feature_columns = list(self.level0_models.keys()) + [
            'mean_usage', 'peak_ratio', 'zero_ratio', 
            'weekend_effect', 'peak_hour_concentration'
        ]
        
        X = features_df[feature_columns].copy()
        X = X.fillna(0)
        
        # 정규화
        X_scaled = self.scaler.transform(X)
        
        # 예측
        predictions = self.level1_model.predict(X_scaled)
        
        # 음수 방지
        predictions = np.maximum(predictions, 0)
        
        return predictions
    
    def evaluate_model(self, X_test, y_test):
        """모델 성능 평가"""
        if not self.is_fitted:
            raise ValueError("❌ 모델이 훈련되지 않았습니다.")
        
        # 예측
        X_test_scaled = self.scaler.transform(X_test)
        y_pred = self.level1_model.predict(X_test_scaled)
        y_pred = np.maximum(y_pred, 0)  # 음수 방지
        
        # 성능 지표 계산
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # 상대 오차 계산
        relative_errors = np.abs((y_test - y_pred) / (y_test + 1e-8))
        mape = np.mean(relative_errors) * 100
        
        print("📊 모델 성능 평가:")
        print(f"  MAE (평균절대오차): {mae:.4f}")
        print(f"  MSE (평균제곱오차): {mse:.4f}")
        print(f"  RMSE (제곱근평균제곱오차): {rmse:.4f}")
        print(f"  R² (결정계수): {r2:.4f}")
        print(f"  MAPE (평균절대백분율오차): {mape:.2f}%")
        
        return {
            'mae': mae,
            'mse': mse,
            'rmse': rmse,
            'r2': r2,
            'mape': mape
        }
    
    def detect_anomalous_volatility(self, features_df, threshold_percentile=95):
        """
        이상 변동성 탐지
        
        Parameters:
        features_df: 특성 데이터프레임
        threshold_percentile: 이상치 기준 백분위수
        
        Returns:
        anomaly_results: 이상치 탐지 결과
        """
        if not self.is_fitted:
            raise ValueError("❌ 모델이 훈련되지 않았습니다.")
        
        print("🚨 이상 변동성 탐지 중...")
        
        # 변동계수 예측
        predicted_volatility = self.predict(features_df)
        
        # 이상치 기준 설정
        threshold = np.percentile(predicted_volatility, threshold_percentile)
        
        # 이상치 탐지
        anomaly_mask = predicted_volatility > threshold
        
        anomaly_results = pd.DataFrame({
            'customer_id': features_df['customer_id'],
            'predicted_volatility': predicted_volatility,
            'is_anomaly': anomaly_mask,
            'anomaly_score': (predicted_volatility - threshold) / threshold
        })
        
        anomaly_customers = anomaly_results[anomaly_results['is_anomaly']]
        
        print(f"🎯 탐지 결과:")
        print(f"  전체 고객: {len(anomaly_results)}명")
        print(f"  이상 변동성 고객: {len(anomaly_customers)}명")
        print(f"  이상치 비율: {len(anomaly_customers)/len(anomaly_results)*100:.1f}%")
        
        if len(anomaly_customers) > 0:
            print(f"\n⚠️ 주의 필요 고객:")
            for _, row in anomaly_customers.sort_values('anomaly_score', ascending=False).iterrows():
                print(f"  {row['customer_id']}: 변동계수 {row['predicted_volatility']:.3f} (점수: {row['anomaly_score']:.2f})")
        
        return anomaly_results
    
    def generate_volatility_report(self, features_df):
        """종합 변동성 리포트 생성"""
        if not self.is_fitted:
            raise ValueError("❌ 모델이 훈련되지 않았습니다.")
        
        print("\n" + "="*60)
        print("📋 전력 사용패턴 변동계수 종합 리포트")
        print("="*60)
        
        # 변동계수 예측
        predicted_volatility = self.predict(features_df)
        
        # 통계 요약
        print("\n📊 변동계수 분포:")
        print(f"  평균: {np.mean(predicted_volatility):.3f}")
        print(f"  표준편차: {np.std(predicted_volatility):.3f}")
        print(f"  최소값: {np.min(predicted_volatility):.3f}")
        print(f"  최대값: {np.max(predicted_volatility):.3f}")
        print(f"  중위수: {np.median(predicted_volatility):.3f}")
        
        # 등급별 분류
        p33 = np.percentile(predicted_volatility, 33)
        p67 = np.percentile(predicted_volatility, 67)
        
        print(f"\n🏷️ 변동성 등급:")
        print(f"  안정형 (< {p33:.3f}): {np.sum(predicted_volatility < p33)}명")
        print(f"  보통형 ({p33:.3f} ~ {p67:.3f}): {np.sum((predicted_volatility >= p33) & (predicted_volatility < p67))}명")
        print(f"  변동형 (≥ {p67:.3f}): {np.sum(predicted_volatility >= p67)}명")
        
        # 고객별 결과
        results_df = pd.DataFrame({
            'customer_id': features_df['customer_id'],
            'predicted_volatility': predicted_volatility,
            'actual_cv': features_df['traditional_cv'] if 'traditional_cv' in features_df.columns else np.nan
        })
        
        results_df['grade'] = pd.cut(results_df['predicted_volatility'], 
                                   bins=[0, p33, p67, np.inf], 
                                   labels=['안정형', '보통형', '변동형'])
        
        print(f"\n👥 고객별 변동계수:")
        print("고객번호\t예측변동계수\t등급")
        for _, row in results_df.iterrows():
            print(f"{row['customer_id']}\t{row['predicted_volatility']:.3f}\t\t{row['grade']}")
        
        # 이상 변동성 탐지
        anomaly_results = self.detect_anomalous_volatility(features_df)
        
        print(f"\n💡 활용 방안:")
        print("  1. 비정상적 전력 사용 패턴 조기 감지")
        print("  2. 고객별 맞춤형 전력 관리 서비스")
        print("  3. 영업 리스크 평가 및 관리")
        print("  4. 전력 수요 예측 정확도 향상")
        
        return {
            'predictions': results_df,
            'anomalies': anomaly_results,
            'statistics': {
                'mean': np.mean(predicted_volatility),
                'std': np.std(predicted_volatility),
                'percentiles': {'p33': p33, 'p67': p67}
            }
        }

# 사용 예제 및 테스트
def create_sample_lp_data():
    """테스트용 샘플 LP 데이터 생성"""
    import random
    from datetime import datetime, timedelta
    
    data = []
    customers = [f'A{1001+i}' for i in range(10)]
    start_date = datetime(2024, 3, 1)
    
    # 다양한 변동성 패턴의 고객들
    volatility_patterns = {
        'A1001': 0.3,   # 안정적
        'A1002': 1.2,   # 높은 변동성
        'A1003': 0.25,  # 매우 안정적
        'A1004': 0.8,   # 중간 변동성
        'A1005': 0.9,   # 중간 변동성
        'A1006': 0.2,   # 안정적
        'A1007': 1.5,   # 매우 높은 변동성
        'A1008': 0.7,   # 중간 변동성
        'A1009': 0.6,   # 중간 변동성
        'A1010': 0.4    # 안정적
    }
    
    for customer in customers:
        target_cv = volatility_patterns[customer]
        base_power = random.uniform(50, 150)
        
        for day in range(31):
            current_date = start_date + timedelta(days=day)
            
            for hour in range(24):
                for minute in [0, 15, 30, 45]:
                    timestamp = current_date.replace(hour=hour, minute=minute)
                    
                    # 변동성에 따른 노이즈 생성
                    noise_std = base_power * target_cv
                    power = base_power + random.gauss(0, noise_std)
                    power = max(0, power)
                    
                    data.append({
                        '대체고객번호': customer,
                        'LP수신일자': timestamp.strftime('%Y-%m-%d-%H:%M'),
                        '순방향유효전력': round(power, 1)
                    })
    
    return pd.DataFrame(data)

if __name__ == "__main__":
    print("🚀 한국전력공사 전력 사용패턴 변동계수 스태킹 알고리즘 테스트")
    print("="*70)
    
    # 1. 샘플 데이터 생성
    print("1️⃣ 샘플 데이터 생성...")
    lp_data = create_sample_lp_data()
    print(f"   생성 완료: {len(lp_data):,}레코드")
    
    # 2. 스태킹 모델 초기화
    print("\n2️⃣ 스태킹 모델 초기화...")
    stacking_model = KEPCOVolatilityStackingModel()
    
    # 3. 특성 추출
    print("\n3️⃣ 변동성 특성 추출...")
    features_df = stacking_model.extract_features(lp_data)
    
    # 4. 훈련 데이터 준비
    print("\n4️⃣ 훈련 데이터 준비...")
    X, y = stacking_model.prepare_training_data(features_df)
    
    # 5. 모델 훈련
    print("\n5️⃣ 스태킹 모델 훈련...")
    stacking_model.fit(X, y)
    
    # 6. 종합 리포트 생성
    print("\n6️⃣ 종합 변동성 리포트 생성...")
    report = stacking_model.generate_volatility_report(features_df)
    
    print("\n🎯 스태킹 알고리즘 구현 완료!")
    print("   • Level-0: 7개 기본 변동성 모델")
    print("   • Level-1: 최적화된 메타모델")
    print("   • 과적합 방지: 교차검증 + 정규화")
    print("   • 이상 탐지: 자동 임계값 설정")

🚀 한국전력공사 전력 사용패턴 변동계수 스태킹 알고리즘 테스트
1️⃣ 샘플 데이터 생성...
   생성 완료: 29,760레코드

2️⃣ 스태킹 모델 초기화...

3️⃣ 변동성 특성 추출...
🔧 변동성 특성 추출 중...
✅ 특성 추출 완료: 10명 고객, 13개 특성

4️⃣ 훈련 데이터 준비...
📊 훈련 데이터 준비 완료: 10개 샘플, 12개 특성

5️⃣ 스태킹 모델 훈련...
🚀 스태킹 모델 훈련 시작...
🔍 메타모델 성능 비교:
모델명		MAE		MSE		R²
linear      	0.0210		0.0005		nan
ridge       	0.0275		0.0009		nan
lasso       	0.1296		0.0240		nan
elastic     	0.0803		0.0081		nan
rf          	0.0958		0.0119		nan
gbm         	0.0795		0.0069		nan


ValueError: ❌ 적합한 메타모델을 찾을 수 없습니다.